In [ ]:
from src.IOSystem import IOSystem
from src.SupplyChain import SupplyChain

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
database = IOSystem(year=2022, language="german").load()

In [ ]:
S = database.Impact.S.copy().to_numpy()
L = database.L.copy().to_numpy()
Y = database.Y.copy().to_numpy()
LY = (database.L @ database.Y).to_numpy()

In [ ]:
region_names = database.regions
impact_names = database.impacts

In [ ]:
region_indices = {region_names[j]:[i+200*j for i in range(200)] for j in range(49)}

In [ ]:
# Anzahl der Regionen (49 Regionen mit je 200 Sektoren)
num_regions = len(database.regions)
sectors_per_region = len(database.sectors)  # Anzahl der Sektoren pro Region

# Liste, in der die Umweltauswirkungs-Matrizen jeder Region gespeichert werden
all_region_impacts = []

for region in range(num_regions):
    # Bestimme Start- und Endindex der aktuellen Region
    start = region * sectors_per_region
    end = (region + 1) * sectors_per_region
    print(start, end)
    
    # Extrahiere den Teil von L und S, der zur aktuellen Region gehört:
    L_region = L[start:end, :]    # Dimension: (200, 9800)
    S_region = S[:, start:end]      # Dimension: (126, 200)
    
    # Berechne den Produktionsvektor für die Sektoren dieser Region:
    # x_region zeigt, wie viel in den 200 Sektoren dieser Region produziert werden muss,
    # um die Endnachfrage aus allen 9800 Sektoren zu bedienen.
    x_region = L_region @ Y         # Dimension: (200, 9800)
    
    # Berechne die in der Region entstehenden Umweltauswirkungen:
    # Dabei wird die Umweltintensitätsmatrix S_region auf die regionale Produktion angewendet.
    E_region = S_region @ x_region  # Dimension: (126, 9800)
    
    # Speichere die resultierende 126x9800 Matrix
    all_region_impacts.append(E_region)

# Füge alle Regionen untereinander:
final_impacts = np.vstack(all_region_impacts)  # Dimension: (126 * 49, 9800) = (6174, 9800)

print(final_impacts.shape)

In [ ]:
df = pd.DataFrame(final_impacts, columns=database.Index.sector_multiindex, index=database.Index.impact_per_region_multiindex)

In [ ]:
df

In [ ]:
idx = pd.IndexSlice

id = df.loc[
    idx["Treibhausgasemissionen", :],  # alle Regionen unter diesem Impact
    idx[:, "Deutschland", :, :, :, :, "Textilien"]  # alle Spalten mit dieser Kombination
]


In [ ]:
id.sum()

In [ ]:
def multiindex_to_nested_dict(multiindex: pd.MultiIndex) -> dict:
    """Convert a MultiIndex to a nested dictionary for hierarchical QTreeWidget use."""
    root = {}
    for keys in multiindex:
        current = root
        for key in keys:
            current = current.setdefault(key, {})
    return root

sector_hierarchy = multiindex_to_nested_dict(database.Index.sector_multiindex_per_region)

In [ ]:
sector_hierarchy

In [ ]:
supplychain = SupplyChain(database=database, Kontinent="Europa", Sector="Textilien")
supplychain

In [ ]:
supplychain = SupplyChain(database=database, select=True)

In [ ]:
supplychain

In [ ]:
supplychain.plot_supply_chain(impacts=["Treibhausgasemissionen", "Wasserverbrauch", "Landnutzung", "Wertschöpfung", "Arbeitszeit"])

In [ ]:
supplychain.calculate_all(impacts=["Treibhausgasemissionen", "Wasserverbrauch", "Landnutzung", "Wertschöpfung", "Arbeitszeit"], relative=False, decimal_places=5)

In [ ]:
supplychain.plot_subcontractors(color="Greens")

In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTabWidget, QWidget, QVBoxLayout, QHBoxLayout,
    QGroupBox, QLabel, QComboBox, QPushButton, QSplitter, QTreeWidget,
    QTreeWidgetItem, QScrollArea, QApplication
)
from PyQt5.QtCore import Qt

class UserInterface(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        self.setWindowTitle("Exiobase Explorer")
        self.resize(800, 450)

        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)

        central_layout = QVBoxLayout(central_widget)
        central_layout.setContentsMargins(20, 20, 20, 20)
        central_layout.setSpacing(20)

        self.tab_widget = QTabWidget(self)
        self.create_selection_tab()
        central_layout.addWidget(self.tab_widget)

        self._create_menu_bar()
        self.show()

    def create_selection_tab(self):
        selection_tab = QWidget()
        selection_layout = QVBoxLayout(selection_tab)
        selection_layout.setSpacing(20)

        # === General Settings ===
        general_settings_group = QGroupBox("General Settings")
        gen_layout = QHBoxLayout(general_settings_group)
        self.language_combo = QComboBox()
        self.language_combo.addItems(["English", "Deutsch", "Français", "Español"])
        self.year_combo = QComboBox()
        self.year_combo.addItems(["2021", "2022", "2023", "2024"])
        gen_layout.addWidget(QLabel("Language:"))
        gen_layout.addWidget(self.language_combo)
        gen_layout.addWidget(QLabel("Year:"))
        gen_layout.addWidget(self.year_combo)
        selection_layout.addWidget(general_settings_group)

        # === Region/Sector Widget ===
        region_sector_widget = QWidget()
        rs_layout = QHBoxLayout(region_sector_widget)
        rs_layout.setSpacing(20)
        rs_layout.setContentsMargins(0, 0, 0, 0)

        # Region Selection with QTreeWidget
        region_group = QGroupBox("Region Selection")
        region_layout = QVBoxLayout(region_group)
        self.region_tree = QTreeWidget()
        self.region_tree.setHeaderHidden(True)

        # Kontinente und Länder
        region_hierarchy = {
            "Europa": ["Germany", "France", "Spain", "Italy", "Poland"],
            "Asien": ["China", "Japan", "India"],
            "Nordamerika": ["USA", "Canada", "Mexico"],
        }
        for cont, countries in region_hierarchy.items():
            parent = QTreeWidgetItem(self.region_tree)
            parent.setText(0, cont)
            parent.setFlags(parent.flags() | Qt.ItemIsUserCheckable)
            parent.setCheckState(0, Qt.Unchecked)
            for country in countries:
                child = QTreeWidgetItem(parent)
                child.setText(0, country)
                child.setFlags(child.flags() | Qt.ItemIsUserCheckable)
                child.setCheckState(0, Qt.Unchecked)

        self.region_tree.itemChanged.connect(self.on_region_item_changed)
        region_layout.addWidget(self.region_tree)
        btn_clear_region = QPushButton("Clear Region Selection")
        btn_clear_region.clicked.connect(self.clear_region_selection)
        region_layout.addWidget(btn_clear_region)

        # Sector Selection with hierarchical QTreeWidget
        sector_group = QGroupBox("Sector Selection")
        sector_layout = QVBoxLayout(sector_group)

        self.sector_tree = QTreeWidget()
        self.sector_tree.setHeaderHidden(True)


        def add_tree_items(parent, data):
            for key, val in data.items():
                item = QTreeWidgetItem(parent)
                item.setText(0, key)
                item.setFlags(item.flags() | Qt.ItemIsUserCheckable)
                item.setCheckState(0, Qt.Unchecked)
                if isinstance(val, dict) and val:
                    add_tree_items(item, val)

        add_tree_items(self.sector_tree, sector_hierarchy)
        self.sector_tree.itemChanged.connect(self.on_sector_item_changed)

        sector_layout.addWidget(self.sector_tree)
        btn_clear_sector = QPushButton("Clear Sector Selection")
        btn_clear_sector.clicked.connect(self.clear_sector_selection)
        sector_layout.addWidget(btn_clear_sector)

        rs_layout.addWidget(region_group)
        rs_layout.addWidget(sector_group)

        # === Bottom Widget (Summary + Buttons) ===
        bottom_widget = QWidget()
        bottom_layout = QVBoxLayout(bottom_widget)
        bottom_layout.setSpacing(10)
        bottom_layout.setContentsMargins(0, 0, 0, 0)

        self.summary_group = QGroupBox("Selection Summary")
        summary_layout = QVBoxLayout(self.summary_group)
        self.selection_label = QLabel("No selection made")
        self.selection_label.setWordWrap(True)
        summary_scroll = QScrollArea()
        summary_scroll.setWidgetResizable(True)
        summary_scroll.setWidget(self.selection_label)
        summary_layout.addWidget(summary_scroll)
        bottom_layout.addWidget(self.summary_group)

        btn_layout = QHBoxLayout()
        self.apply_button = QPushButton("Apply Selection")
        self.apply_button.clicked.connect(self.apply_selection)
        self.reset_button = QPushButton("Reset All Selections")
        self.reset_button.clicked.connect(self.reset_selection)
        btn_layout.addWidget(self.apply_button)
        btn_layout.addWidget(self.reset_button)
        bottom_layout.addLayout(btn_layout)

        splitter = QSplitter(Qt.Vertical)
        splitter.setContentsMargins(0, 0, 0, 0)
        splitter.addWidget(region_sector_widget)
        splitter.addWidget(bottom_widget)
        splitter.setStretchFactor(0, 1)
        splitter.setStretchFactor(1, 1)

        selection_layout.addWidget(splitter)
        self.tab_widget.addTab(selection_tab, "Selection")

    def propagate_down(self, item, state):
        # Rekursive Funktion, um alle Nachkommen zu setzen
        for i in range(item.childCount()):
            child = item.child(i)
            child.setCheckState(0, state)
            self.propagate_down(child, state)

    def on_region_item_changed(self, item, column):
        state = item.checkState(column)
        self.region_tree.blockSignals(True)
        self.propagate_down(item, state)
        self.region_tree.blockSignals(False)
        self.update_summary()

    def on_sector_item_changed(self, item, column):
        state = item.checkState(column)
        self.sector_tree.blockSignals(True)
        self.propagate_down(item, state)
        self.sector_tree.blockSignals(False)
        self.update_summary()

    def clear_region_selection(self):
        root = self.region_tree.invisibleRootItem()
        for i in range(root.childCount()):
            continent = root.child(i)
            continent.setCheckState(0, Qt.Unchecked)
            self.propagate_down(continent, Qt.Unchecked)
        self.update_summary()

    def clear_sector_selection(self):
        root = self.sector_tree.invisibleRootItem()
        for i in range(root.childCount()):
            parent = root.child(i)
            parent.setCheckState(0, Qt.Unchecked)
            self.propagate_down(parent, Qt.Unchecked)
        self.update_summary()

    def get_checked_regions(self):
        checked = []
        root = self.region_tree.invisibleRootItem()
        for i in range(root.childCount()):
            continent = root.child(i)
            for j in range(continent.childCount()):
                country = continent.child(j)
                if country.checkState(0) == Qt.Checked:
                    checked.append(country.text(0))
        return checked

    def get_checked_sectors(self):
        checked = []
        root = self.sector_tree.invisibleRootItem()
        def recurse(item):
            for i in range(item.childCount()):
                child = item.child(i)
                if child.checkState(0) == Qt.Checked and child.childCount() == 0:
                    checked.append(child.text(0))
                recurse(child)
        recurse(root)
        return checked

    def apply_selection(self):
        lang = self.language_combo.currentText()
        yr = self.year_combo.currentText()
        selected_regions = self.get_checked_regions()
        selected_sectors = self.get_checked_sectors()
        self.selection_label.setText(
            f"Selection applied!<br><b>Language:</b> {lang}, <b>Year:</b> {yr}<br>"
            f"<b>Regions:</b> {', '.join(selected_regions)}<br>"
            f"<b>Sectors:</b> {', '.join(selected_sectors)}"
        )
        self.summary_group.setTitle("Selection Summary (saved)")

    def reset_selection(self):
        self.clear_region_selection()
        self.clear_sector_selection()
        self.selection_label.setText("No selection made")
        self.summary_group.setTitle("Selection Summary")

    def update_summary(self):
        selected_regions = self.get_checked_regions()
        selected_sectors = self.get_checked_sectors()
        if not selected_regions and not selected_sectors:
            self.selection_label.setText("No selection made")
        else:
            txt = ""
            if selected_regions:
                txt += f"<b>Regions:</b> {', '.join(selected_regions)}<br>"
            if selected_sectors:
                txt += f"<b>Sectors:</b> {', '.join(selected_sectors)}"
            self.selection_label.setText(txt)

    def _create_menu_bar(self):
        self.menuBar()

if __name__ == "__main__":
    app = QApplication.instance() or QApplication(sys.argv)
    window = UserInterface()
    window.show()
